<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Bayesian_Neural_Networks_(BNNs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal, kl_divergence

# Define the BayesianLinear class
class BayesianLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(BayesianLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight_mu = nn.Parameter(torch.zeros(out_features, in_features))  # Mean of weights
        self.weight_rho = nn.Parameter(torch.ones(out_features, in_features))  # Log-standard deviation of weights
        self.bias_mu = nn.Parameter(torch.zeros(out_features))  # Mean of biases
        self.bias_rho = nn.Parameter(torch.ones(out_features))  # Log-standard deviation of biases

    def forward(self, x):
        weight_sigma = torch.log1p(torch.exp(self.weight_rho))  # Convert log-std to std
        bias_sigma = torch.log1p(torch.exp(self.bias_rho))  # Convert log-std to std

        weight = Normal(self.weight_mu, weight_sigma).rsample()  # Sample weights from normal distribution
        bias = Normal(self.bias_mu, bias_sigma).rsample()  # Sample biases from normal distribution

        return torch.matmul(x, weight.t()) + bias  # Linear transformation

# Define the BayesianNN class
class BayesianNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BayesianNN, self).__init__()
        self.fc1 = BayesianLinear(input_dim, 64)
        self.fc2 = BayesianLinear(64, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Apply ReLU activation
        return self.fc2(x)

# Define the ELBO loss function
def elbo_loss(model, x, y, kl_weight=0.01):
    preds = model(x)
    mse_loss = nn.MSELoss()(preds, y)  # Mean Squared Error loss
    kl_loss = sum(kl_divergence(Normal(param.weight_mu, torch.log1p(torch.exp(param.weight_rho))),
                                 Normal(0, 1)).sum() for param in model.modules() if isinstance(param, BayesianLinear))  # KL divergence loss
    return mse_loss + kl_weight * kl_loss  # Combined ELBO loss

# Example usage
model = BayesianNN(input_dim=10, output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Generate random training data
x_train = torch.randn(100, 10)
y_train = torch.randn(100, 1)

# Train the model
for epoch in range(100):
    optimizer.zero_grad()
    loss = elbo_loss(model, x_train, y_train)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item():.4f}')